In [1]:
## setup
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf

conf = pyspark.SparkConf().setAppName('Lab1').setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [4]:
## get the file into hdfs
db = sc.textFile("data/mat.txt")

In [5]:
db.collect()

['A,1,1,1',
 'A,1,2,-2',
 'A,1,3,-3',
 'A,1,4,1',
 'A,2,1,4',
 'A,2,2,-5',
 'A,2,3,-1',
 'A,2,4,3',
 'A,3,1,-7',
 'A,3,2,8',
 'A,3,3,-9',
 'A,3,4,-1',
 'B,1,1,1',
 'B,1,2,2',
 'B,1,3,3',
 'B,1,4,1',
 'B,1,5,0',
 'B,2,1,4',
 'B,2,2,5',
 'B,2,3,1',
 'B,2,4,3',
 'B,2,5,-9',
 'B,3,1,8',
 'B,3,2,8',
 'B,3,3,-9',
 'B,3,4,-1',
 'B,3,5,10',
 'B,4,1,0',
 'B,4,2,4',
 'B,4,3,9',
 'B,4,4,3',
 'B,4,5,2']

#### a) To Check if Aand B are compatible to be added. Output "TRUE" or "False"

In [16]:
rdd1 = db.flatMap(lambda a: [(a.split(',')[0]+'i', int(a.split(',')[1])), (a.split(',')[0]+'j', int(a.split(',')[2]))]    )
rdd2 = rdd1.reduceByKey(lambda a,b:max(a,b))
rdd3 = rdd2.map(lambda a: (a[0][1], a[1])).reduceByKey(lambda a,b:a==b).map(lambda a:('Output', a[1])).reduceByKey(lambda a,b: a and b)
rdd3.collect()

[('Output', False)]

#### b) If so, add them to produce C=A+B

In [23]:
rdd1 = db.map(lambda a: (('C', a.split(',')[1], a.split(',')[2]), int(a.split(',')[3]))).reduceByKey(lambda a,b: a+b)
rdd2 = rdd1.map(lambda a: ','.join(a[0])+','+str(a[1]) )
rdd2.collect()

['C,1,1,2',
 'C,1,2,0',
 'C,1,3,0',
 'C,1,4,2',
 'C,2,1,8',
 'C,2,2,0',
 'C,2,3,0',
 'C,2,4,6',
 'C,3,1,1',
 'C,3,2,16',
 'C,3,3,-18',
 'C,3,4,-2',
 'C,1,5,0',
 'C,2,5,-9',
 'C,3,5,10',
 'C,4,1,0',
 'C,4,2,4',
 'C,4,3,9',
 'C,4,4,3',
 'C,4,5,2']

#### c) To check if A and B are compatible to be multiplied. Output "TRUE" or "FALSE"

In [29]:
rdd1 = db.map(lambda a: ('Aj', int(a.split(',')[2])) if a.split(',')[0]=='A' else ('Bi', int(a.split(',')[1])) )
rdd2 = rdd1.reduceByKey(lambda a,b: max(a,b)).map(lambda a: ('Output',a[1])).reduceByKey(lambda a,b:a==b)
rdd2.collect()

[('Output', True)]

#### d) If so, multiply them to produce C=AB

In [52]:
rdd1 = db.map(lambda a: (a.split(',')[2], [('A', a.split(',')[1], int(a.split(',')[3]))] ) if a.split(',')[0]=='A' else  (a.split(',')[1], [('B', a.split(',')[2], int(a.split(',')[3]))] ) )
rdd2 = rdd1.reduceByKey(lambda a,b:a+b).map(lambda a:( a[0],[ [ x for x in a[1] if x[0]=='A'], [ x for x in a[1] if x[0]=='B']]) )
rdd3 = rdd2.flatMap(lambda a: [(('C', x[1],y[1]) ,x[2]*y[2]) for x in a[1][0] for y in a[1][1] ]).reduceByKey(lambda a,b:a+b)
rdd4 = rdd3.map(lambda a: ','.join(a[0])+','+str(a[1]))
rdd4.collect()

['C,1,1,-31',
 'C,1,2,-28',
 'C,1,3,37',
 'C,1,4,1',
 'C,1,5,-10',
 'C,2,1,-24',
 'C,2,2,-13',
 'C,2,3,43',
 'C,2,4,-1',
 'C,2,5,41',
 'C,3,1,-47',
 'C,3,2,-50',
 'C,3,3,59',
 'C,3,4,23',
 'C,3,5,-164']